In [1]:
# Download the model
! python /mnt/code/lit-gpt/scripts/download.py --repo_id tiiuae/falcon-7b

Fetching 5 files: 100%|███████████████████████████| 5/5 [01:51<00:00, 22.40s/it]


In [2]:
# Convert the checkpoint
! sudo python /mnt/code/lit-gpt/scripts/convert_hf_checkpoint.py --checkpoint_dir checkpoints/tiiuae/falcon-7b

Model config {'org': 'tiiuae', 'name': 'falcon-7b', 'block_size': 2048, 'vocab_size': 50254, 'padding_multiple': 512, 'padded_vocab_size': 65024, 'n_layer': 32, 'n_head': 71, 'n_embd': 4544, 'rotary_percentage': 1.0, 'parallel_residual': True, 'bias': False, 'n_query_groups': 1, 'shared_attention_norm': True, '_norm_class': 'LayerNorm', 'norm_eps': 1e-05, '_mlp_class': 'GptNeoxMLP', 'intermediate_size': 18176, 'condense_ratio': 1}
Processing checkpoints/tiiuae/falcon-7b/pytorch_model-00001-of-00002.bin
Processing checkpoints/tiiuae/falcon-7b/pytorch_model-00002-of-00002.bin


In [3]:
# Prepare the instruction dataset for training
! python /mnt/code/lit-gpt/scripts/prepare_alpaca.py \--checkpoint_dir checkpoints/tiiuae/falcon-7b/

Loading data file...
Loading tokenizer...
train has 198 samples
test has 7 samples
Processing train split ...
{'instruction': "It's a bit challenging, but I'm getting the hang of it.", 'context': "I've started learning coding. \n\n Interesting! Which programming language are you starting with? \n\n That's true, Python is quite user-friendly. How are you finding it?", 'response': "Keep at it. It'll start to make more sense with time."}
100%|███████████████████████████████████████| 198/198 [00:00<00:00, 1170.37it/s]
Processing test split ...
100%|███████████████████████████████████████████| 7/7 [00:00<00:00, 1487.12it/s]


In [ ]:
# Fine tune the model
# This will take a while to run, please go to the python file to adjust parameters for training
! python /mnt/code/lit-gpt/finetune/lora.py \
  --checkpoint_dir checkpoints/tiiuae/falcon-7b/ \
  --precision bf16-true

In [6]:
# Generate output from the fine tuned model , change the precision according to the GPU you are using
! python /mnt/code/lit-gpt/generate/lora.py --checkpoint_dir '/mnt/code/checkpoints/tiiuae/falcon-7b' \
  --lora_path '/mnt/artifacts/out/lora/alpaca/lit_model_lora_finetuned_instruct.pth' \
  --max_new_tokens 300 \
  --precision bf16-true

Loading model '/mnt/code/checkpoints/tiiuae/falcon-7b/lit_model.pth' with {'org': 'tiiuae', 'name': 'falcon-7b', 'block_size': 2048, 'vocab_size': 50254, 'padding_multiple': 512, 'padded_vocab_size': 65024, 'n_layer': 32, 'n_head': 71, 'n_embd': 4544, 'rotary_percentage': 1.0, 'parallel_residual': True, 'bias': False, 'n_query_groups': 1, 'shared_attention_norm': True, '_norm_class': 'LayerNorm', 'norm_eps': 1e-05, '_mlp_class': 'GptNeoxMLP', 'intermediate_size': 18176, 'condense_ratio': 1, 'r': 16, 'alpha': 32, 'dropout': 0.05, 'to_query': False, 'to_key': False, 'to_value': False, 'to_projection': False, 'to_mlp': False, 'to_head': False}
Time to instantiate model: 1.35 seconds.
Time to load the model weights: 7.40 seconds.
You (I) are chatting with a user R. Write a reply to their message.

### Your previous communication:


### Their new message:
What food do lamas eat?

### Your response:
R: bananas

### Their new message:
How much bananas do lamas eat?

### Your response:
R: as m

In [7]:
# Generate output from the base model that has not been fine tuned
! python /mnt/code/lit-gpt/generate/base.py --checkpoint_dir '/mnt/code/checkpoints/tiiuae/falcon-7b' \
    --prompt 'What food do lamas eat?' \
    --max_new_tokens 300 \
    --precision bf16-true

Loading model '/mnt/code/checkpoints/tiiuae/falcon-7b/lit_model.pth' with {'org': 'tiiuae', 'name': 'falcon-7b', 'block_size': 2048, 'vocab_size': 50254, 'padding_multiple': 512, 'padded_vocab_size': 65024, 'n_layer': 32, 'n_head': 71, 'n_embd': 4544, 'rotary_percentage': 1.0, 'parallel_residual': True, 'bias': False, 'n_query_groups': 1, 'shared_attention_norm': True, '_norm_class': 'LayerNorm', 'norm_eps': 1e-05, '_mlp_class': 'GptNeoxMLP', 'intermediate_size': 18176, 'condense_ratio': 1}
Time to instantiate model: 1.35 seconds.
Time to load the model weights: 7.26 seconds.
Seed set to 1234
What food do lamas eat?
Lamas eat whatever their human hosts eat, but they are not allowed to eat meat and dairy products. The following is a list of some of the foods that Tibetan lamas eat: noodles, dairy products, eggs, meat and vegetables.[Note : This list is not comprehensive]
What is the diet of a monk?
A monk's diet consists mainly of rice and vegetables. However, he may drink tea and water

In [8]:
# Quantize the base model and generate output
# Please check the amount of GPU memory available and restart the kernel at this point or clear GPU memory to avoid OOM errors
! python /mnt/code/lit-gpt/generate/base.py --quantize bnb.int8 \
    --checkpoint_dir 'checkpoints/tiiuae/falcon-7b' \
    --prompt 'What food do lamas eat?' \
    --precision bf16-true \
    --max_new_tokens 300


Loading model 'checkpoints/tiiuae/falcon-7b/lit_model.pth' with {'org': 'tiiuae', 'name': 'falcon-7b', 'block_size': 2048, 'vocab_size': 50254, 'padding_multiple': 512, 'padded_vocab_size': 65024, 'n_layer': 32, 'n_head': 71, 'n_embd': 4544, 'rotary_percentage': 1.0, 'parallel_residual': True, 'bias': False, 'n_query_groups': 1, 'shared_attention_norm': True, '_norm_class': 'LayerNorm', 'norm_eps': 1e-05, '_mlp_class': 'GptNeoxMLP', 'intermediate_size': 18176, 'condense_ratio': 1}
Time to instantiate model: 3.77 seconds.
Time to load the model weights: 9.94 seconds.
Seed set to 1234
What food do lamas eat?
Lamas eat whatever their human caretakers eat. Unfortunately for the Tibetan Sheepdogs, the diet of lamas is not what they would prefer. It is thought that lamas eat one-third dairy products and two-thirds grains, although we have no way of knowing exactly what they eat.
How much does a lama eat?
According to the Lama Temple, a typical Tibetan lama eats about 6-12 lbs. of hay and 8-1

In [11]:
# Quantize the fine tuned model and generate output
! python /mnt/code/lit-gpt/generate/lora.py --quantize bnb.int8 \
    --checkpoint_dir 'checkpoints/tiiuae/falcon-7b' \
    --prompt 'What food do llamas eat?' \
    --precision bf16-true \
    --max_new_tokens 300

Loading model 'checkpoints/tiiuae/falcon-7b/lit_model.pth' with {'org': 'tiiuae', 'name': 'falcon-7b', 'block_size': 2048, 'vocab_size': 50254, 'padding_multiple': 512, 'padded_vocab_size': 65024, 'n_layer': 32, 'n_head': 71, 'n_embd': 4544, 'rotary_percentage': 1.0, 'parallel_residual': True, 'bias': False, 'n_query_groups': 1, 'shared_attention_norm': True, '_norm_class': 'LayerNorm', 'norm_eps': 1e-05, '_mlp_class': 'GptNeoxMLP', 'intermediate_size': 18176, 'condense_ratio': 1, 'r': 16, 'alpha': 32, 'dropout': 0.05, 'to_query': False, 'to_key': False, 'to_value': False, 'to_projection': False, 'to_mlp': False, 'to_head': False}
Time to instantiate model: 2.07 seconds.
Time to load the model weights: 9.73 seconds.
You (I) are chatting with a user R. Write a reply to their message.

### Your previous communication:


### Their new message:
What food do llamas eat?

### Your response:
Hi R, thanks for chatting. Llamas are grazers and eat a variety of plants. They often graze on grass, 